# Phi-4 Mini ONNX Rinnakkaisten Funktiokutsujen Opas

Tämä notebook esittelee, kuinka käyttää Phi-4 Miniä ONNX Runtime GenAI:n kanssa rinnakkaisiin funktiokutsuihin. Funktiokutsut mahdollistavat mallin älykkään ulkoisten työkalujen ja API:iden käytön käyttäjän pyyntöjen perusteella.

## Yleiskatsaus

Tässä oppaassa opit:
- Miten asettaa Phi-4 Mini ONNX Runtime GenAI:n kanssa
- Määrittämään funktioskeemat lentojen ja hotellien varaamista varten
- Käyttämään ohjattua generointia Lark-kieliopin avulla rakenteelliseen tulostukseen
- Suorittamaan rinnakkaisia funktiokutsuja monimutkaisissa matkavaraustilanteissa

## Esivaatimukset

Ennen kuin suoritat tämän notebookin, varmista että:
- Olet ladannut Phi-4 Mini ONNX -mallin
- Olet asentanut `onnxruntime-genai`-paketin
- Sinulla on perustiedot funktiokutsujen konsepteista


## Vaihe 1: Tuo tarvittavat kirjastot

Ensiksi tuodaan tarvittavat kirjastot funktiokutsujen toteutusta varten.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Vaihe 2: Mallin asennus ja konfigurointi

Nyt konfiguroidaan Phi-4 Mini ONNX -malli. Muista korvata polku omalla mallihakemistollasi.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Vaihe 3: Määritä generointiparametrit

Aseta generointiparametrit hallitsemaan mallin vastauskäyttäytymistä. Nämä asetukset varmistavat määrätietoiset ja keskittyneet vastaukset funktiokutsuille.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Vaihe 4: Määritä käytettävissä olevat toiminnot

Tässä määritellään toiminnot, joita tekoälyavustajamme voi käyttää. Tässä esimerkissä meillä on kaksi matkailuun liittyvää toimintoa:
1. **booking_flight_tickets**: Lentolippujen varaamiseen lentokenttien välillä
2. **booking_hotels**: Hotellimajoitusten varaamiseen

Toimintojen määrittelyt noudattavat OpenAI:n toimintokutsujen skeemamuotoa.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Vaihe 5: Kieliopin luontiaputoiminnot

Nämä apufunktiot muuntavat funktioiden määritelmät Lark-kieliopin muotoon, jota ONNX Runtime GenAI käyttää ohjattuun generointiin. Tämä varmistaa, että malli tuottaa kelvollisia funktiokutsuja oikeassa JSON-muodossa.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Vaihe 6: Testaa kieliopin luontia

Testataan kieliopin luontitoimintoja ja katsotaan, miten ne muuntavat työkalumääritelmämme oikeaan muotoon.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Vaihe 7: Valmistele järjestelmän kehotus ja generaattori

Nyt luomme järjestelmän kehotuksen, joka kertoo mallille käytettävissä olevista työkaluista, ja asetamme generaattorin ohjattujen generointiparametrien avulla.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Vaihe 8: Alusta generaattori ohjatulla generoinnilla

Nyt luomme generaattorin määritetyillä parametreilla ja sovellamme Lark-kielioppia ohjattua generointia varten.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Vaihe 9: Testaa rinnakkaisten funktioiden kutsumista

Testataan nyt asetusta monimutkaisella matkavarauksen skenaariolla, joka vaatii useiden funktioiden kutsumista.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Luo Funktiokutsuja

Malli tuottaa nyt jäsenneltyjä funktiokutsuja käyttäjän pyynnön perusteella. Ohjatun generoinnin ansiosta tulos on kelvollisessa JSON-muodossa, joka voidaan suorittaa suoraan.

**Odotettu tulos**: Mallin tulisi tuottaa funktiokutsut seuraaville:
1. `booking_flight_tickets` sisältäen tiedot Pekingistä (PEK) Pariisiin (CDG)
2. `booking_hotels` sisältäen tiedot majoituksesta Pariisissa

Suorita alla oleva solu nähdäksesi reaaliaikaisen generoinnin:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Johtopäätös

🎉 **Onnittelut!** Olet onnistuneesti toteuttanut rinnakkaisen funktiokutsun Phi-4 Mini -mallilla käyttäen ONNX Runtime GenAI:ta.

### Mitä olet oppinut:

1. **Mallin asetukset**: Kuinka konfiguroida Phi-4 Mini ONNX Runtime GenAI:n kanssa
2. **Funktioiden määrittely**: Kuinka määritellä funktioiden skeemat tekoälyn funktiokutsuja varten
3. **Ohjattu generointi**: Kuinka käyttää Lark-grammatiikkaa rakenteellisen tulostuksen tuottamiseen
4. **Rinnakkaiset funktiokutsut**: Kuinka käsitellä monimutkaisia pyyntöjä, jotka vaativat useita funktiokutsuja

### Keskeiset hyödyt:

- ✅ **Rakenteellinen tulostus**: Ohjattu generointi varmistaa validit JSON-funktiokutsut
- ✅ **Rinnakkaiskäsittely**: Mahdollisuus käsitellä useita funktiokutsuja yhdellä pyynnöllä
- ✅ **Korkea suorituskyky**: ONNX Runtime tarjoaa optimoitua inferenssiä
- ✅ **Joustava skeema**: Helppo lisätä tai muokata funktiomäärittelyjä

### Resurssit:

- [Phi-4 Mini Dokumentaatio](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI Dokumentaatio](https://onnxruntime.ai/docs/genai/)
- [Funktiokutsujen parhaat käytännöt](https://platform.openai.com/docs/guides/function-calling)



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäinen asiakirja sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
